In [1]:
import boto3

In [2]:
from easydict import EasyDict as ed

config = ed(
    region_name = 'us-west-2',
    s3_bucket_name = 'steem-hackaton-input'
)

## Create an S3 Bucket

In [11]:
s3 = boto3.client('s3', region_name=config.region_name)

In [12]:
from funcy import where, first
exists = first(where(s3.list_buckets()['Buckets'], Name=config['s3_bucket_name']))
if not exists:
    response = s3.create_bucket(
        ACL='public-read',
        Bucket=config.s3_bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': config.region_name,
        },
    )
    assert response['HTTPStatusCode'] == 200

## Upload to S3

In [33]:
import pathlib

filename = 'amy.png'
input_file = pathlib.Path.home() / 'Downloads' / filename

result = s3.upload_file(
    Filename=str(input_file), 
    Bucket=config.s3_bucket_name, 
    Key=f'{filename}',
)

## Rekognition API

In [3]:
rkg = boto3.client('rekognition', region_name=config.region_name)

In [4]:
## NSFW
response = rkg.detect_moderation_labels(
    Image={
        'S3Object': {
            'Bucket': config.s3_bucket_name,
            'Name': 'amy.png',
        }
    },
)
response['ModerationLabels']

[{'Confidence': 50.02201843261719,
  'Name': 'Explicit Nudity',
  'ParentName': ''},
 {'Confidence': 50.02201843261719,
  'Name': 'Nudity',
  'ParentName': 'Explicit Nudity'}]

In [5]:
# CELEBRITIES
response = rkg.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': config.s3_bucket_name,
            'Name': 'amy.png',
        }
    },
)
response

{'CelebrityFaces': [],
 'OrientationCorrection': 'ROTATE_0',
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '729',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 01 Nov 2017 18:43:44 GMT',
   'x-amzn-requestid': '95e4969f-bf34-11e7-b076-475a8b9a6c2d'},
  'HTTPStatusCode': 200,
  'RequestId': '95e4969f-bf34-11e7-b076-475a8b9a6c2d',
  'RetryAttempts': 0},
 'UnrecognizedFaces': [{'BoundingBox': {'Height': 0.390625,
    'Left': 0.4735576808452606,
    'Top': 0.24519230425357819,
    'Width': 0.38942307233810425},
   'Confidence': 99.87974548339844,
   'Landmarks': [{'Type': 'eyeLeft',
     'X': 0.6886886954307556,
     'Y': 0.40068140625953674},
    {'Type': 'eyeRight', 'X': 0.7786025404930115, 'Y': 0.5228366851806641},
    {'Type': 'nose', 'X': 0.6861751675605774, 'Y': 0.5471510291099548},
    {'Type': 'mouthLeft', 'X': 0.5608097314834595, 'Y': 0.5201963186264038},
    {'Type': 'mouthRight', 'X': 0.6328566670417786, 'Y': 0.6108995

In [ ]:
# rkg.get_celebrity_info(Id='string:celebrity_id_form_prev_request')

In [9]:
response = rkg.detect_labels(
    Image={
        'S3Object': {
            'Bucket': config.s3_bucket_name,
            'Name': 'amy.png',
        }
    },
    MaxLabels=100,
    MinConfidence=80,
)
response['Labels']

[{'Confidence': 99.1627197265625, 'Name': 'People'},
 {'Confidence': 99.16273498535156, 'Name': 'Person'},
 {'Confidence': 99.1453857421875, 'Name': 'Human'}]